# TVB-NEST: Bridging multiscale activity by co-simulation

## Step-by-step learn how to perform a co-simulation embedding spiking neural networks into large-scale brain networks using TVB.

In [ ]:
from IPython.core.display import Image, display
display(Image(filename='pics/ConceptGraph1.png',  width=1000, unconfined=False))

## tvb-multiscale toolbox:

### https://github.com/the-virtual-brain/tvb-multiscale

For questions use the git issue tracker, or write an e-mail to me: dionysios.perdikis@charite.de

# TVB - NEST co-simulation 

## Reduced Wong-Wang TVB mean field model

For every region node $n\prime$ modelled as a mean-field node in TVB:

(Post)Synaptic gating dynamics (i.e., proportion of synapse channels open at any given time):

$\dot{S_{n\prime}} = - \frac{1}{\tau}{S_{n\prime}}(t) + (1-{S_{n\prime}}(t))\gamma {R_{n\prime}}(t)$

and $ {R_{n\prime}}(t) $ is the postsynaptic firing rate given by:

$ {R_{n\prime}}(t) = H({I_{syn_{n\prime}}}(t), a, b, d) $

where

$ H({I_{syn_{n\prime}}}(t),  a, b, d) = \frac{aI_{syn_{n\prime}}(t)-b}{1-e^{-d(a{I_{syn_{n\prime}}}(t)-b)}}$ 

is a sigmoidal activation function of the input presynaptic current.

The total input presynaptic current to excitatory populations is given by: 

$ {I_{syn_{n\prime}}}(t) = I_o + w_+J_{N}{S_{n\prime}}(t) + GJ_{N}\sum_{{m\prime}\neq {n\prime}}C_{{m\prime}{n\prime}}S_{m\prime}(t-\tau_{{m\prime}{n\prime}})$

## Reduced Wong-Wang mean field model

### Parameters following Deco et al 2013:

- structural TVB connectivity weights $C_{{m\prime}{n\prime}}$ (${m\prime}->{n\prime}$)
- structural TVB connectivity delays $\tau_{{m\prime}{n\prime}}$  (${m\prime}->{n\prime}$)
- global structural brain connectivity coupling constant $G$
- overall effective external input current $I_o = 0.3nA$ 
- excitatory synaptic coupling $J_{N} = 0.2609nA$ 
- local excitatory recurrence $w_+ = 0.9$
- excitatory kinetic parameter $\gamma = 0.641 s$
- excitatory sigmoidal functions parameters $a = 2710nC^{-1}$, $b = 108Hz$, $d = 0.154s$


## Izhikevich Spiking network model in NEST

For every neuron $i$ in region node $n$ modelled in NEST as a spiking network:

Membrane potential:

$ \dot{V}_m = n_2V_m^2 + n_1V_m + n_0140 - U_m/C - g_{AMPA}(V_m-E_{AMPA}) - g_{GABA}(V_m-E_{GABA}) - g_{BASE}V_m + I_e$

where the conductances follow the equations:

$ \dot{g}_{AMPA} = - g_{AMPA} / \tau_{AMPA} + \left[\sum_k \delta(t-t_k) \right]_{Exc}$

$ \dot{g}_{GABA} = - g_{GABA} / \tau_{GABA} + \left[\sum_k \delta(t-t_k) \right]_{Inh}$

$ \dot{g}_{BASE} = - g_{BASE} / \tau_{BASE} + \left[\sum_k \delta(t-t_k) \right]_{BASE}$

and recovery variable:

$ \dot{U}_m = a(bV_m - U_m)$


When $ V_m > V_{th} $ , $ V_m $ is set to $ c $, and $ U_m $ is incremented by $ d $.

## TVB to NEST coupling
TVB couples to NEST via instantaneous spike rate $ interface_{weight} * R(t) $, 

Spike generator NEST devices are used as TVB "proxy" nodes and generate spike trains 

$ \left[ \sum_k \delta(t-\tau_{n\prime n}-{t_j}^k) \right]_{j \in n\prime} $



In [ ]:
display(Image(filename='pics/Rate_BG.png',  width=1000, unconfined=False))

## NEST to TVB update

A NEST spike detector device is used to count spike for each time step, and convert it to an instantaneous population mean rate that overrides an auxiliary TVB state variables $R_{in}(t)$, which drives a linear integration equation of another auxiliary TVB state variable $R_{int}(t)$, which,in its turn, acts as a smoothing low pass filter:

$ \dot{R}_{int_{n}}  = -\frac{1}{\tau_{rin_{n}}}(R_{int_{n}}(t) - {R_{in_{n}}}(t)) $


where:

$ {R_{in_{n}}}(t) =  \frac{\sum_j\left[ \sum_k \delta(t-\tau_n-{t_j}^k) \right]_{j \in R_n}}{nNeurons * dt} $ in  spikes/sec.

Finally, $ {R_{int_{n}}}(t) $ overwrites the variable ${R_{n}}(t)$ via a user defined transform function:

${R_{n}}(t) = f_{NEST->TVB}({R_{int_{n}}}(t)) $

This update process concerns only the TVB region nodes that are simulated exclusively in NEST, as spiking networks. All the rest of TVB nodes will follow the equations of the mean field model described above.


In [ ]:
display(Image(filename='pics/NESTtoTVB_BG.png',  width=1000, unconfined=False))

# WORKFLOW:

In [ ]:
import os
from collections import OrderedDict
import time
import numpy as np

from tvb.basic.profile import TvbProfile
TvbProfile.set_profile(TvbProfile.LIBRARY_PROFILE)

INTERFACE_COUPLING_MODE = "spikeNet" # "TVB"  or "spikeNet"
data_mode = "patient" # "control", or "patient"

from tvb_multiscale.tvb_nest.config import *

work_path = os.getcwd()
outputs_path = os.path.join(work_path, "outputs/Izhikevich")
sim_mode_path = os.path.join(outputs_path, "TVBcortex_no_opt")
config = Config(output_base=sim_mode_path)

config.figures.SHOW_FLAG = True 
config.figures.SAVE_FLAG = True
config.figures.FIG_FORMAT = 'png'
config.figures.DEFAULT_SIZE= config.figures.NOTEBOOK_SIZE
FIGSIZE = config.figures.DEFAULT_SIZE

data_path = os.path.join(config.DATA_PATH, 'basal_ganglia')
fit_data_path = os.path.join(data_path, "ANNarchyFittedModels/dataFits_2020_02_05/databestfits", )
control_data = os.path.join(fit_data_path, "controlleft/OutputSim_Patient08.mat")
patient_data = os.path.join(fit_data_path, "patientleft/OutputSim_Patient09.mat")

if data_mode == "patient":
    subject_data = patient_data
    if INTERFACE_COUPLING_MODE == "TVB":
        STN_factor = 20.5 * 1e-2  # 0.205 
        dSN_factor = 22.2 * 1e-2  # 0.222 
        iSN_factor = 20.2 * 1e-2
    else:
        STN_factor = 30.05 * 1e-2
        dSN_factor = 17.64 * 1e-2
        iSN_factor = 20.24 * 1e-2
else:
    subject_data = control_data
    if INTERFACE_COUPLING_MODE == "TVB":
        STN_factor = 20.2 * 1e-2  # 0.202 
        dSN_factor = 23.4 * 1e-2  # 0.2340 
        iSN_factor = 21.1 * 1e-2  # 0.211 
    else:
        STN_factor = 24.92 * 1e-2
        dSN_factor = 14.9 * 1e-2
        iSN_factor = 18.53 * 1e-2

simulation_length = 500.0
transient = 10.0
init_cond_jitter = 0.0

SPIKING_NODES_DELAYS = False

from tvb_multiscale.core.plot.plotter import Plotter
plotter = Plotter(config.figures)

# For interactive plotting:
# %matplotlib notebook  

# Otherwise:
%matplotlib inline 

## 1. Load structural data <br> (minimally a TVB connectivity)  <br> & prepare TVB simulator  <br> (region mean field model, integrator, monitors etc)

In [ ]:
from tvb_multiscale.core.tvb.cosimulator.models.reduced_wong_wang_exc_io import ReducedWongWangExcIO

# ----------------------------------------------------------------------------------------------------------------
# ----Uncomment below to modify the simulator by changing the default options:--------------------------------------
# ----------------------------------------------------------------------------------------------------------------

from tvb.datatypes.connectivity import Connectivity
from tvb.simulator.integrators import HeunStochastic
from tvb.simulator.monitors import Raw  # , Bold, EEG
    

# 0. GPe_Left, 1. GPi_Left, 2. STN_Left, 3. Striatum_Left, 4. Thal_Left
BG_opt_matrix_weights = np.zeros((5, 5))
conn_mode = "subject" # subject" or "average"
if conn_mode == "average":
    weights_maith = np.array([1.93, 3.56, 1.46, 4.51, 3.52, 2.30, 2.34, 3.78, 1.98, 
                             1.30, 1.82, 3.56, 3.02, 1.78, 1.36, 2.27, 4.13, 2.74, 3.27])*1e-3  # controls
#     weights_maith = np.array([3.27, 3.80, 2.65, 3.66, 3.06, 3.06, 3.25, 4.02, 3.32, 
#                             2.98, 3.45, 3.64, 2.50, 2.12, 2.86, 2.79, 3.96, 3.69, 3.87])*1e-3   # patients
    # probs_maith = ????
else:
    import scipy.io as sio
    weights=sio.loadmat(subject_data)    # weights start from index 19
    weights_maith = weights["X"][0, 19:] # these are indices 19 till 37
    probs_maith = weights["X"][0, :19]   # these are indices 0 till 18

wdSNGPi = BG_opt_matrix_weights[3, 1] = weights_maith[0].item()
wiSNGPe = BG_opt_matrix_weights[3, 0] = weights_maith[1].item()
wGPeSTN = BG_opt_matrix_weights[0, 2] = weights_maith[2].item()
wSTNGPe = BG_opt_matrix_weights[2, 0] = weights_maith[3].item()
wSTNGPi = BG_opt_matrix_weights[2, 1] = weights_maith[4].item()
wGPeGPi = BG_opt_matrix_weights[0, 1] = weights_maith[5].item()  
wGPiTh = BG_opt_matrix_weights[1, 4] = weights_maith[8].item()
wThdSN = BG_opt_matrix_weights[4, 3] = weights_maith[10].item() # Th -> dSN
    
sliceBGnet = slice(0,5)

wGPeGPe = weights_maith[6].item()   # "GPe" -> "GPe" 
wGPiGPi = weights_maith[7].item()   # "GPi" -> "GPi" 
wThiSN = weights_maith[9].item()    # "Eth" -> "IiSN" 

wdSNdSN = weights_maith[11].item()  # "IdSN" -> "IdSN" 
wiSNiSN = weights_maith[12].item()  # "IiSN" -> "IiSN" 
wCtxdSN = weights_maith[13].item()  # "CxE" -> "IdSN" 
wCtxiSN = weights_maith[14].item()  # "CxE" -> "IiSN" 
wCtxSTN = weights_maith[15].item()  # "CxE" -> "Estn"
wCtxEtoI = weights_maith[16].item() # "CxE" -> "CxI"
wCtxItoE = weights_maith[17].item() # "CxI" -> "CxE"
wCtxItoI = weights_maith[18].item() # "CxI" -> "CxI"

pdSNGPi = probs_maith[0].item()
piSNGPe = probs_maith[1].item()
pGPeSTN = probs_maith[2].item()
pSTNGPe = probs_maith[3].item()
pSTNGPi = probs_maith[4].item()
pGPeGPi = probs_maith[5].item()
pGPeGPe = probs_maith[6].item()     # "GPe" -> "GPe" 
pGPiGPi = probs_maith[7].item()     # "GPi" -> "GPi" 
pGPiTh = probs_maith[8].item()
pThiSN =  probs_maith[9].item()     # "Eth" -> "IiSN
pThdSN = probs_maith[10].item()     # Th --> dSN
pdSNdSN = probs_maith[11].item()    # "IdSN" -> "IdSN" 
piSNiSN = probs_maith[12].item()    # "IiSN" -> "IiSN" 
pCtxdSN = probs_maith[13].item()    # "CxE" -> "IdSN" 
pCtxiSN = probs_maith[14].item()    # "CxE" -> "IiSN" 
pCtxSTN = probs_maith[15].item()    # "CxE" -> "Estn"
pCtxEtoI = probs_maith[16].item()   # "CxE" -> "CxI"
pCtxItoE = probs_maith[17].item()   # "CxI" -> "CxE"
pCtxItoI = probs_maith[18].item()   # "CxI" -> "CxI"
pCtxCtx = probs_maith[16:19].mean() # "Ctx" -> "Ctx"

In [ ]:
display(Image(filename='pics/Connectome.png',  width=1000, unconfined=False))

In [ ]:
# Load full TVB connectome connectivity

conn_path = os.path.join(data_path, "conn")

#Load AAL atlas normative connectome including the Basal Ganglia regions from Petersen et al. atlas
wTVB = np.loadtxt(os.path.join(conn_path, "conn_denis_weights.txt"))
cTVB = np.loadtxt(os.path.join(conn_path, "aal_plus_BG_centers.txt"),usecols=range(1,4))
rlTVB = np.loadtxt(os.path.join(conn_path, "aal_plus_BG_centers.txt"),dtype="str", usecols=(0,))
tlTVB = np.loadtxt(os.path.join(conn_path, "BGplusAAL_tract_lengths.txt"))

# Remove the second Thalamus, Pallidum (GPe/i), Putamen and Caudate (Striatum):
inds_Th = (rlTVB.tolist().index("Thalamus_L"), rlTVB.tolist().index("Thalamus_R"))
inds_Pall = (rlTVB.tolist().index("Pallidum_L"), rlTVB.tolist().index("Pallidum_R"))
inds_Put = (rlTVB.tolist().index("Putamen_L"), rlTVB.tolist().index("Putamen_R"))
inds_Caud = (rlTVB.tolist().index("Caudate_L"), rlTVB.tolist().index("Caudate_R"))
inds_rm = inds_Th + inds_Pall + inds_Put + inds_Caud
print("Connections of Thalami, Pallidum (GPe/i), Putamen and Caudate (Striatum) removed!:\n", 
      wTVB[inds_rm, :][:, inds_rm])
wTVB = np.delete(wTVB, inds_rm, axis=0)
wTVB = np.delete(wTVB, inds_rm, axis=1)
tlTVB = np.delete(tlTVB, inds_rm, axis=0)
tlTVB = np.delete(tlTVB, inds_rm, axis=1)
rlTVB = np.delete(rlTVB, inds_rm, axis=0)
cTVB = np.delete(cTVB, inds_rm, axis=0)

number_of_regions = len(rlTVB)
speed = np.array([4.0])
min_tt = speed.item() * 0.1
sliceBG = [0, 1, 2, 3, 6, 7]
sliceCortex = slice(10, number_of_regions)

# Remove BG -> Cortex connections
print("Removing BG -> Cortex connections with max:")
print(wTVB[sliceCortex, :][:, sliceBG].max())
wTVB[sliceCortex, sliceBG] = 0.0
tlTVB[sliceCortex, sliceBG] = min_tt

# Remove Cortex -> Thalamus connections
sliceThal = [8, 9]
print("Removing Cortex -> Thalamus connections with summed weight:")
print(wTVB[sliceThal, sliceCortex].sum())
wTVB[sliceThal, sliceCortex] = 0.0
tlTVB[sliceThal, sliceCortex] = min_tt

# Remove Cortex -> GPe/i connections
sliceGP = [0, 1, 2, 3]
print("Removing Cortex -> GPe/i connections with max:")
print(wTVB[sliceGP, sliceCortex].max())
wTVB[sliceGP,  sliceCortex] = 0.0
tlTVB[sliceGP, sliceCortex] = min_tt

# # Minimize all delays for the optimized network
# tlTVB[:7][:, :7] = min_tt

connTVB = Connectivity(region_labels=rlTVB, weights=wTVB, centres=cTVB, tract_lengths=tlTVB, speed=speed)

# Normalize connectivity weights
connTVB.weights = connTVB.scaled_weights(mode="region")
connTVB.weights /= np.percentile(connTVB.weights, 99)

# Keep only left hemisphere and remove Vermis:
sliceLeft = slice(0, connTVB.number_of_regions -8, 2)

connLeft = Connectivity(region_labels=connTVB.region_labels[sliceLeft], 
                        centres=connTVB.centres[sliceLeft],
                        weights=connTVB.weights[sliceLeft][:, sliceLeft],
                        tract_lengths=connTVB.tract_lengths[sliceLeft][:, sliceLeft], 
                        speed=connTVB.speed)
connLeft.configure()

print("\nLeft cortex connectome, after removing direct BG -> Cortex and intehemispheric BG <-> BG connections:")
plotter.plot_tvb_connectivity(connLeft);

In [ ]:
sliceBGnet = slice(0,5)
connTVBleftBG = Connectivity(region_labels=connLeft.region_labels[sliceBGnet], 
                             centres=connLeft.centres[sliceBGnet],
                             weights=connLeft.weights[sliceBGnet][:, sliceBGnet],
                             tract_lengths=connLeft.tract_lengths[sliceBGnet][:, sliceBGnet], 
                            speed=connLeft.speed)
connTVBleftBG.configure()

print("\nLeft BG TVB network:")
plotter.plot_tvb_connectivity(connTVBleftBG);

In [ ]:
scaleBGoptTOtvb = np.percentile(BG_opt_matrix_weights, 95) /\
                  np.percentile(connTVBleftBG.weights, 95)
                  
print("Scaling factor of TVB BG network connectome to optimal one = %g" % scaleBGoptTOtvb)

In [ ]:
# Construct the final connectivity to use for simulation:
# Rescale the 
ww = scaleBGoptTOtvb * np.array(connLeft.weights)
ww[sliceBGnet, sliceBGnet] = BG_opt_matrix_weights.T  # !!!NOTE TVB indices convention Wi<-j!!!

connectivity = Connectivity(region_labels=connLeft.region_labels, 
                            centres=connLeft.centres,
                            weights=ww, tract_lengths=connLeft.tract_lengths, 
                            speed=connLeft.speed)
connectivity.configure()

# Construct only the optimized BG connectivity only for plotting:
connBGopt = Connectivity(region_labels=connectivity.region_labels[sliceBGnet], 
                         centres=connectivity.centres[sliceBGnet],
                         weights=connectivity.weights[sliceBGnet][:, sliceBGnet],
                         tract_lengths=connectivity.tract_lengths[sliceBGnet][:, sliceBGnet], 
                         speed=connectivity.speed)
connBGopt.configure()

print("\nLeft BG optimized network:")
plotter.plot_tvb_connectivity(connBGopt);

In [ ]:
from tvb_multiscale.core.tvb.cosimulator.cosimulator_serial import CoSimulatorSerial as CoSimulator

#white_matter_coupling = coupling.Linear(a=0.014)
# Create a TVB simulator and set all desired inputs
# (connectivity, model, surface, stimuli etc)
# We choose all defaults in this example
simulator = CoSimulator()
#simulator.use_numba = False
model_params = {"G": np.array([15.0/scaleBGoptTOtvb])}
simulator.model = ReducedWongWangExcIO(**model_params)

simulator.connectivity = connectivity

simulator.integrator = HeunStochastic()
simulator.integrator.dt = 0.1
simulator.integrator.noise.nsig = np.array([1e-4])  # 1e-5

mon_raw = Raw(period=1.0)  # ms
simulator.monitors = (mon_raw, )

init_cond_filepath = os.path.join(data_path, "tvb_init_cond_left.npy")
init_cond = np.load(init_cond_filepath)   # 
init_cond = np.abs(init_cond *(1 + init_cond_jitter * np.random.normal(size=init_cond.shape)))
simulator.connectivity.set_idelays(simulator.integrator.dt)
simulator.initial_conditions = init_cond * np.ones((simulator.connectivity.idelays.max(),
                                                    simulator.model.nvar,
                                                    simulator.connectivity.number_of_regions,
                                                    simulator.model.number_of_modes))

simulator.configure()


print("\nConnectome used for simulations:")
plotter.plot_tvb_connectivity(simulator.connectivity);


# # Serializing TVB cosimulator is necessary for parallel cosimulation:
# from tvb_multiscale.core.utils.file_utils import dump_pickled_dict
# from tvb_multiscale.core.tvb.cosimulator.cosimulator_serialization import serialize_tvb_cosimulator
# sim_serial_filepath = os.path.join(config.out.FOLDER_RES, "tvb_serial_cosimulator.pkl")
# sim_serial = serialize_tvb_cosimulator(simulator)
# display(sim_serial)

# # Dumping the serialized TVB cosimulator to a file will be necessary for parallel cosimulation.
# dump_pickled_dict(sim_serial, sim_serial_filepath)

In [ ]:

display(Image(filename='pics/Network.png',  width=1000, unconfined=False))

## 2. Build and connect the NEST network model <br> (networks of spiking neural populations for fine-scale <br>regions, stimulation devices, spike detectors etc)

In [ ]:
from tvb_multiscale.tvb_nest.nest_models.models.basal_ganglia_izhikevich import BasalGangliaIzhikevichBuilder

# Build a NEST network model with the corresponding builder
from tvb_multiscale.tvb_nest.nest_models.builders.nest_factory import load_nest, configure_nest_kernel

# Load NEST and use defaults to configure its kernel:
nest = configure_nest_kernel(load_nest(config=config), config)

# Select the regions for the fine scale modeling with NEST spiking networks
number_of_regions = simulator.connectivity.region_labels.shape[0]
#including cortex node:
nest_nodes_ids = [0, 1, 2, 3, 4]  #, 5 the indices of fine scale regions modeled with NEST

# Build a NEST network model with the corresponding builder
nest_model_builder = BasalGangliaIzhikevichBuilder(tvb_simulator=simulator,
                                                    spiking_nodes_inds=nest_nodes_ids,
                                                    spiking_simulator=nest,
                                                    config=config
                                                  )

# Using all default parameters for this example
# nest_model_builder.set_defaults()

# or modify the builder by changing the default options:--------------------------------------

population_neuron_model = "izhikevich_hamker"

nest_model_builder.population_order = 200 # reduce for speed

# When any of the properties model, params and scale below depends on regions,
# set a handle to a function with
# arguments (region_index=None) returning the corresponding property

from copy import deepcopy
nest_model_builder.params_common = \
    {"V_m": -70.0, "U_m": -18.55, "E_rev_AMPA": 0.0, "E_rev_GABA_A": -90.0, "V_th": 30.0, "V_r": 0.0, "c": -65.0,
     "C_m": 1.0, "I_e": 0.0, "current_stimulus_scale": -200, "current_stimulus_mode": 2,
     "tau_rise": 1.0, # doesn't seem to have any effect
     "tau_rise_AMPA": 10.0, "tau_rise_GABA_A": 10.0,
     "n0": 140.0, "n1": 5.0, "n2": 0.04}

nest_model_builder._paramsI = deepcopy(nest_model_builder.params_common)
nest_model_builder._paramsI.update({"a": 0.005, "b": 0.585, "d": 4.0})
nest_model_builder._paramsE = deepcopy(nest_model_builder.params_common)
nest_model_builder.paramsStr = deepcopy(nest_model_builder.params_common)
nest_model_builder.paramsStr.update({"V_th": 40.0, "C_m": 50.0, "V_r": -80.0,
                                     "n0": 61.65119, "n1": 2.594639, "n2": 0.022799, 
                                     "a": 0.05, "b": -20.0, "c": -55.0, "d": 377.0})

nest_model_builder.Igpe_nodes_ids = [0]
nest_model_builder.Igpi_nodes_ids = [1]
nest_model_builder.Estn_nodes_ids = [2]
nest_model_builder.Eth_nodes_ids = [4]
nest_model_builder.Istr_nodes_ids = [3]

I_nodes_ids = nest_model_builder.Igpe_nodes_ids + nest_model_builder.Igpi_nodes_ids 
E_nodes_ids = nest_model_builder.Estn_nodes_ids + nest_model_builder.Eth_nodes_ids 


def paramsE_fun(node_id):
    paramsE = deepcopy(nest_model_builder._paramsE)
    if node_id in nest_model_builder.Estn_nodes_ids:
        paramsE.update({"a": 0.005, "b": 0.265, "d": 2.0, "I_e": 3.0})  # dictionary of params for Estn
    elif node_id in nest_model_builder.Eth_nodes_ids:
        paramsE.update({"a": 0.02, "b": 0.25, "d": 0.05, "I_e": 3.5}) # dictionary of params for Eth
    return paramsE
    
def paramsI_fun(node_id):
    # For the moment they are identical, unless you differentiate the noise parameters
    paramsI = deepcopy(nest_model_builder._paramsI)
    if node_id in nest_model_builder.Igpe_nodes_ids:
        paramsI.update({"I_e": 12.0})  # 12.0
    elif node_id in nest_model_builder.Igpi_nodes_ids:
        paramsI.update({"I_e": 30.0})  # 30.0
    return paramsI
    
# Populations' configurations
# When any of the properties model, params and scale below depends on regions,
# set a handle to a function with
# arguments (region_index=None) returning the corresponding property
nest_model_builder.populations = [
    {"label": "E", "model": population_neuron_model,  
     "params":  paramsE_fun, 
     "nodes": E_nodes_ids,  # Estn in [2], Eth in [4], Cortex in [5]
     "scale": 1.0}, # lambda node_id: 3.0 if node_id in nest_model_builder.Crtx_nodes_ids else 
    {"label": "I", "model": population_neuron_model,  
     "params": paramsI_fun, 
     "nodes": I_nodes_ids,  # Igpe in [0], Igpi in [1], Cortex in [5]
     "scale": 1.0}, # lambda node_id: 0.75 if node_id in nest_model_builder.Crtx_nodes_ids else 
    {"label": "IdSN", "model": population_neuron_model,   
     "params": nest_model_builder.paramsStr, 
     "nodes": nest_model_builder.Istr_nodes_ids,  # IdSN in [3]
     "scale": 1.0},
    {"label": "IiSN", "model": population_neuron_model,   # IiSN in [3]
     "params": nest_model_builder.paramsStr, 
     "nodes": nest_model_builder.Istr_nodes_ids,  # None means "all"
     "scale": 1.0}
]

# Within region-node connections
# When any of the properties model, conn_spec, weight, delay, receptor_type below
# set a handle to a function with
# arguments (region_index=None) returning the corresponding property

synapse_model = "static_synapse"
conn_spec = {"allow_autapses": True, "allow_multapses": True, "rule": "all_to_all"}

def conn_spec_fixed_prob(prob=None):
    output = conn_spec.copy()
    if prob is not None:
        output["rule"] = "fixed_total_number"
        output["p"] = prob
    return output

within_node_delay = 1.0

# for each connection, we have a different probability
nest_model_builder.populations_connections = [
     #        source   ->   target
    {"source": "I", "target": "I",  # I -> I This is a self-connection for population "Igpe"
     "synapse_model": synapse_model, "conn_spec": conn_spec_fixed_prob(pGPeGPe),  # conn_spec
     "weight": -np.abs(wGPeGPe).item(), "delay": within_node_delay,
     "nodes": nest_model_builder.Igpe_nodes_ids},  # None means apply to all
    {"source": "I", "target": "I",  # I -> I This is a self-connection for population "Igpi"
     "synapse_model": synapse_model, "conn_spec": conn_spec_fixed_prob(pGPiGPi),  # conn_spec
     "weight": -np.abs(wGPiGPi).item(), "delay": within_node_delay,
     "nodes": nest_model_builder.Igpi_nodes_ids},  # None means apply to all
    {"source": "IdSN", "target": "IdSN",  # IdSN -> IdSN This is a self-connection for population "IdSN"
     "synapse_model": synapse_model, "conn_spec": conn_spec_fixed_prob(pdSNdSN),  # conn_spec
     "weight": -np.abs(wdSNdSN).item(), "delay": within_node_delay,
       "nodes": nest_model_builder.Istr_nodes_ids},
    {"source": "IiSN", "target": "IiSN",  # IiSN -> IiSN This is a self-connection for population "IiSN"
     "synapse_model": synapse_model, "conn_spec": conn_spec_fixed_prob(piSNiSN),  # conn_spec
     "weight": -np.abs(wiSNiSN).item(), "delay": within_node_delay,
     "nodes": nest_model_builder.Istr_nodes_ids},
    ]


# Among/Between region-node connections
# Given that only the AMPA population of one region-node couples to
# all populations of another region-node,
# we need only one connection type
        
# When any of the properties model, conn_spec, weight, delay, receptor_type below
# depends on regions, set a handle to a function with
# arguments (source_region_index=None, target_region_index=None)

from tvb_multiscale.core.spiking_models.builders.templates import scale_tvb_weight, tvb_delay

# We set global coupling scaling to 1.0,
# because we need the Maith et al optimized weights without any scaling:
nest_model_builder.global_coupling_scaling = 1.0
        
# Function that will return the TVB weight with optional scaling:
class TVBWeightFun(object):
    tvb_weights = nest_model_builder.tvb_weights
    global_coupling_scaling = nest_model_builder.global_coupling_scaling
    sign = 1

    def __init__(self, sign=1, scale=nest_model_builder.global_coupling_scaling):
        self.sign = sign
        self.global_coupling_scaling = scale * self.sign
        
    def __call__(self, source_node, target_node):
        return scale_tvb_weight(source_node, target_node, self.tvb_weights,
                                scale=self.global_coupling_scaling)
    
# Function that will return the TVB delay unless SPIKING_NODES_DELAYS == False:
tvb_delay_fun = \
    lambda source_node, target_node: \
        np.maximum(nest_model_builder.tvb_dt, 
                   tvb_delay(source_node, target_node, nest_model_builder.tvb_delays)) \
            if SPIKING_NODES_DELAYS else within_node_delay

def nodes_conn(src_pop, trg_pop, src_nodes, trg_nodes, sign, prob, weight=None):
    if weight is None:
        weight = TVBWeightFun(sign)
    else:
        weight *= sign

    return {
        "source": src_pop, "target": trg_pop,
        "model": synapse_model, "conn_spec": conn_spec_fixed_prob(prob),  # conn_spec
        "weight": weight,
        "delay": lambda source_node, target_node: tvb_delay_fun(source_node, target_node),
        "source_nodes": src_nodes,
        "target_nodes": trg_nodes}

Istr_nodes = nest_model_builder.Istr_nodes_ids
Igpi_nodes = nest_model_builder.Igpi_nodes_ids
Igpe_nodes = nest_model_builder.Igpe_nodes_ids
Eth_nodes = nest_model_builder.Eth_nodes_ids
Estn_nodes = nest_model_builder.Estn_nodes_ids

ampa = 1
gaba = -1

nest_model_builder.nodes_connections = [
    nodes_conn("IdSN", "I",             # "IdSN" -> "Igpi"
               Istr_nodes, Igpi_nodes,
               gaba, pdSNGPi),
    nodes_conn("IiSN", "I",             # "IiSN" -> "Igpe"
               Istr_nodes, Igpe_nodes,
               gaba, piSNGPe),
    nodes_conn("I", "I",                # "Igpe" -> "Igpi"
               Igpe_nodes, Igpi_nodes,
               gaba, pGPeGPi),
    nodes_conn("I", "E",                # "Igpi" -> "Eth"
               Igpi_nodes, Eth_nodes,
               gaba, pGPiTh),
    nodes_conn("I", "E",                # "Igpe" -> "Estn"
               Igpe_nodes, Estn_nodes,
               gaba, pGPeSTN),
    nodes_conn("E", "IdSN",             # "Eth" -> "IdSN"
               Eth_nodes, Istr_nodes,
               ampa, pThdSN, wThdSN),
    nodes_conn("E", "IiSN",             # "Eth" -> "IiSN"
               Eth_nodes, Istr_nodes,
               ampa, pThiSN, wThiSN),
    nodes_conn("E", "I",                # "Estn" -> "Igpe"
               Estn_nodes, Igpe_nodes,
               ampa, pSTNGPe),
    nodes_conn("E", "I",                # "Estn" -> "Igpi"
               Estn_nodes, Igpi_nodes,
               ampa, pSTNGPi),
]


# Creating  devices to be able to observe NEST activity:

nest_model_builder.output_devices = []

#          label <- target population
for pop in nest_model_builder.populations:
    connections = OrderedDict({})
    connections[pop["label"] + "_spikes"] = pop["label"]
    nest_model_builder.output_devices.append(
        {"model": "spike_recorder", "params": {"record_to": "memory"},
         "connections": connections, "nodes": pop["nodes"]})  # None means apply to "all"

# Labels have to be different

connections = OrderedDict({})
#               label    <- target population
params = {"interval": 1.0, "record_to": "memory",
          'record_from': ["V_m", "U_m", "I", "I_syn", "I_syn_ex", "I_syn_in", "g_AMPA", "g_GABA_A", "g_L"]}
for pop in nest_model_builder.populations:
    connections = OrderedDict({})
    connections[pop["label"]] = pop["label"]
    nest_model_builder.output_devices.append(
        {"model": "multimeter", "params": params,
         "connections": connections, "nodes": pop["nodes"]})  # None means apply to all
    
    

#Create a spike stimulus input device
# #including cortex node: we do not need any other external stimulation
# nest_model_builder.Estn_stim = {"rate": 500.0, "weight": 0.009}
# nest_model_builder.Igpe_stim = {"rate": 100.0, "weight": 0.015}
# nest_model_builder.Igpi_stim = {"rate": 700.0, "weight": 0.02}
nest_model_builder.input_devices = [
#     {"model": "poisson_generator",
#      "params": {"rate": nest_model_builder.Estn_stim["rate"], "origin": 0.0, "start": 0.1},
#      "connections": {"BaselineEstn": ["E"]},  # "Estn"
#      "nodes": nest_model_builder.Estn_nodes_ids,  # "Estn"
#      "weights": nest_model_builder.Estn_stim["weight"], "delays": 0.0, "receptor_type": 1},
#     {"model": "poisson_generator",
#      "params": {"rate": nest_model_builder.Igpe_stim["rate"], "origin": 0.0, "start": 0.1},
#      "connections": {"BaselineIgpe": ["I"]},  # "Igpe"
#      "nodes": nest_model_builder.Igpe_nodes_ids,  # "Igpe"
#      "weights": nest_model_builder.Igpe_stim["weight"], "delays": 0.0, "receptor_type": 1},
#     {"model": "poisson_generator",
#      "params": {"rate": nest_model_builder.Igpi_stim["rate"], "origin": 0.0, "start": 0.1},
#      "connections": {"BaselineIgpi": ["I"]},  # "Igpi"
#      "nodes": nest_model_builder.Igpi_nodes_ids,  ## "Igpi"
#      "weights": nest_model_builder.Igpi_stim["weight"], "delays": 0.0, "receptor_type": 1},
#      {"model": "dc_generator",
#      "params": { "amplitude": 1.0, #"frequency": 20.0, "phase": 0.0,"offset": 0.0,
#                 "start": 35.0, "stop": 85.0 },  # "stop": 100.0  "origin": 0.0, 
#      "connections": {"DBS_GPi": ["I"]}, # "GPi"
#      "nodes": nest_model_builder.Igpi_nodes_ids, # "GPi"
#      "weights": 1.0, "delays": 0.0}
#     {"model": "dc_generator",
#      "params": { "amplitude": 1.0, #"frequency": 20.0, "phase": 0.0,"offset": 0.0,
#                 "start": 35.0, "stop": 85.0 },  # "stop": 100.0  "origin": 0.0, 
#      "connections": {"DBS_STN": ["E"]}, # "STN"
#      "nodes": nest_model_builder.Estn_nodes_ids, # "STN"
#      "weights": 1.0, "delays": 0.0}
       ]  #

# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------

nest_model_builder.configure()
nest_network = nest_model_builder.build(set_defaults=False)


In [ ]:
populations_sizes = []
print("Population sizes: ")
for pop in nest_model_builder.populations:
    populations_sizes.append(int(np.round(pop["scale"] * nest_model_builder.population_order)))
    print("%s: %d" % (pop["label"], populations_sizes[-1]))


## 3. Build the TVB-NEST interface

In [ ]:
from tvb_multiscale.tvb_nest.interfaces.models.basal_ganglia_izhikevich import BasalGangliaIzhikevichTVBNESTInterfaceBuilder
from tvb_multiscale.tvb_nest.interfaces.builders import NESTInputProxyModels


# Build a TVB-NEST interface with all the appropriate connections between the
# TVB and NEST modelled regions
tvb_nest_builder = BasalGangliaIzhikevichTVBNESTInterfaceBuilder()
# (simulator, nest_network, nest_nodes_ids, exclusive_nodes=True,populations_sizes=populations_sizes)

tvb_nest_builder.config = config
tvb_nest_builder.tvb_cosimulator = simulator
tvb_nest_builder.spiking_network = nest_network


STN_proxy_inds = np.array(nest_model_builder.Estn_nodes_ids)
Striatum_proxy_inds = np.array(nest_model_builder.Istr_nodes_ids)

# Set exclusive_nodes = True (Default) if the spiking regions substitute for the TVB ones:
tvb_nest_builder.exclusive_nodes = True 


tvb_to_nest_mode = "rate"
nest_to_tvb = True

# If default_coupling_mode = "TVB", large scale coupling towards spiking regions is computed in TVB
# and then applied with no time delay via a single "TVB proxy node" / NEST device for each spiking region,
# "1-to-1" TVB->NEST coupling.
# If any other value, we need 1 "TVB proxy node" / NEST device for each TVB sender region node, and
# large-scale coupling for spiking regions is computed in NEST, 
# taking into consideration the TVB connectome weights and delays, 
# in this "1-to-many" TVB->NEST coupling.
tvb_nest_builder.default_coupling_mode = INTERFACE_COUPLING_MODE # "spikeNet" # "TVB"

tvb_nest_builder.proxy_inds = np.array(nest_nodes_ids)
tvb_nest_builder.N_E = nest_model_builder.population_order

# TVB applies a global coupling scaling of coupling.a * model.G
tvb_nest_builder.global_coupling_scaling = \
    tvb_nest_builder.tvb_cosimulator.coupling.a[0].item() * tvb_nest_builder.G
print("global_coupling_scaling = %g" % tvb_nest_builder.global_coupling_scaling)

# Optionally adjust interface scale factors here 
# to have the same result counter act possible changes to G and coupling.a:
STN_factor /= tvb_nest_builder.global_coupling_scaling
dSN_factor /= tvb_nest_builder.global_coupling_scaling
iSN_factor /= tvb_nest_builder.global_coupling_scaling

# Total TVB indegree weight to STN:
wTVBSTNs = simulator.connectivity.weights[nest_model_builder.Estn_nodes_ids, 5:].squeeze()
wTVBSTN = wTVBSTNs.sum().item()
print("wTVBSTN = %g" % wTVBSTN)
CTXtoSTNinds = 5 + np.where(wTVBSTNs > 0.0)[0] # indices of TVB regions coupling to STN

# Total TVB indegree weight to Striatum:
wTVBSNs = simulator.connectivity.weights[nest_model_builder.Istr_nodes_ids, 5:].squeeze()
wTVBSN = wTVBSNs.sum().item()
print("wTVBSN = %g" % wTVBSN)
CTXtoSNinds = 5 + np.where(wTVBSNs > 0.0)[0]  # indices of TVB regions coupling to Striatum

# Using all default parameters for this example

# or...


# ----------------------------------------------------------------------------------------------------------------
# ----Uncomment below to modify the builder by changing the default options:--------------------------------------
# ----------------------------------------------------------------------------------------------------------------


# --------For spike transmission from TVB to NEST devices acting as TVB proxy nodes with TVB delays:--------


# TVB -> NEST


if tvb_nest_builder.default_coupling_mode == "spikeNet":
    
    # If coupling is computing in NEST, we need as many TVB proxies 
    # as TVB regions coupling to STN and Striatum
    proxy_inds = np.unique(np.concatenate([CTXtoSTNinds, CTXtoSNinds]))
    
    # This is the TVB coupling weight function that will determine the connections' weights 
    # from TVB proxies to the target STN and dSN/iSN populations:
    class TVBWeightFunInterface(object):
    
        def __init__(self, scaling):
            self.scaling = float(scaling)

        def __call__(self, source_node, target_node, tvb_weights):
            return (scale_tvb_weight(source_node, target_node, tvb_weights, scale=self.scaling))

tvb_nest_builder.output_interfaces = []
if tvb_to_nest_mode == "rate":
    # Mean spike rates are applied in parallel to all target neurons
    for trg_pop, target_nodes, conn_scaling, this_conn_spec, scale_factor in \
                                    zip(["E", "IdSN", "IiSN"], # NEST target populations,
                                     [STN_proxy_inds, Striatum_proxy_inds, Striatum_proxy_inds],
                                     [wCtxSTN, wCtxdSN, wCtxiSN], # ...weights
                                     # ...and probabilities for CTX -> STN/Striatum connections
                                     [conn_spec_fixed_prob(prob=pCtxSTN),  # pCtxSTN  
                                      conn_spec_fixed_prob(prob=pCtxdSN),  # pCtxdSN
                                      conn_spec_fixed_prob(prob=pCtxiSN)], # pCtxiSN
                                     # Interface scaling factors scaled by TVB weights' indegree to STN/Striatum:
                                     [STN_factor/wTVBSTN, dSN_factor/wTVBSN, iSN_factor/wTVBSN]
                                     ): # Indices of target region modelled in NEST:
        tvb_nest_builder.output_interfaces.append(
            {"voi": np.array(["R"]), #  Source TVB state variable
             "populations": np.array([trg_pop]), # NEST target population
             "model": "RATE",
             "spiking_proxy_inds": target_nodes, # Target region indices in NEST
             "proxy_model": NESTInputProxyModels.RATE,
             "proxy_params": {
                "allow_offgrid_times": False
             },
             "conn_spec": this_conn_spec,
             "coupling_mode": tvb_nest_builder.default_coupling_mode
        })

        # For both coupling modes, we scale the TVB rate already at the TVB -> NEST transformer
        # with the interface scale factor (normalized by TVB indegree to STN/Striatum)
        # and the global coupling scaling.
        this_interface = tvb_nest_builder.output_interfaces[-1]
        if this_interface["coupling_mode"] == "spikeNet":
            this_interface["proxy_inds"] = proxy_inds
            # In this case connections from each TVB proxy to STN/Striatum 
            # are scaled additionally with the Maith et al. optimized weights
            this_interface["weights"] = TVBWeightFunInterface(conn_scaling)
            this_interface["transformer_params"] = \
                {"scale_factor": scale_factor * tvb_nest_builder.global_coupling_scaling}
            # In total:
            # From each TVB proxy node we get a rate scaled as (
            # (coupling.a * G * STN_factor/wTVBSTN) * R_i, (i for all TVB regions)
            # Then, spikes generated from each TVB proxy are transferred via connections 
            # with weights TVB_w_ji * wCtxSTN or wCtxiSN or wCtxdSN (j for STN or Striatum) 
            # and probabilities pCtxSTN or pCtxiSN or pCtxdSN, respectively
        else:
            # In this case connections from each TVB proxy to STN/Striatum 
            # are equal to the Maith et al. optimized weights
            this_interface["weights"] = conn_scaling
            # In this case coupling.a is already applied during computing TVB coupling.
            # Therefore we scale only with model.G
            this_interface["transformer_params"] = \
                {"scale_factor": scale_factor * tvb_nest_builder.G}
            # In total:
            # From each TVB proxy node we get a total coupling rate scaled 
            # as (coupling.a * G STN_factor/wTVBSTN) * R_j, (j for STN or Striatum)
            # Then, spikes generated from each TVB proxy are transferred via connections 
            # with weights wCtxSTN or wCtxiSN or wCtxdSN and 
            # probabilities pCtxSTN or pCtxiSN or pCtxdSN, respectively

if nest_to_tvb:
    tvb_nest_builder.input_interfaces = []
    # TVB <-- NEST:

    from tvb_multiscale.core.interfaces.transformers.models.red_wong_wang import ElephantSpikesRateRedWongWangExc

    for src_pop, nodes, in zip(
                    # Source populations in NEST:
                    [np.array(["I"]),  np.array(["E"]), np.array(["IdSN", "IiSN"])],
                    # Source regions indices in NEST:
                    [I_nodes_ids,          # GPe and GPi
                    E_nodes_ids,          # STN and Thalamus
                    Striatum_proxy_inds]): # Striatum
            #            TVB <- NEST
            tvb_nest_builder.input_interfaces.append(
                {"voi": np.array(["S", "R"]),  # Target state variables in TVB
                "populations": src_pop,  # Source populations in NEST
                # This transformer not only converts spike counts to rates for state variable R,
                # but also integrates the dS/dt to compute the respective S!:
                "transformer": ElephantSpikesRateRedWongWangExc,
                "transformer_params": 
                    # Spike counts are converted to rates via:
                    # number_of_spikes / number_of_neurons_per_population / number_of_populations
                    # (mind that there are 2 populations in Striatum)
                    {"scale_factor": np.array([1.0]) / tvb_nest_builder.N_E / len(src_pop),
                    # The integrator used to integrate dS/dt
                    "integrator":CONFIGURED.DEFAULT_TRANSFORMER_INTEGRATOR_MODEL(
                                        dt=simulator.integrator.dt),
                        "state": np.zeros((2, len(nodes))), # initial condition
                        # Parameters of the dS/dt differential equation:
                        "tau_s": simulator.model.tau_s, # time constant of integrating S
                        "tau_r": np.array([10.0]),      # time constant of integrating R to low pass filter it
                        "gamma": simulator.model.gamma}, 
                "proxy_inds": np.array(nodes)})  # None means all here

tvb_nest_builder.w_tvb_to_spike_rate = 1.0
# We return from a NEST spike_recorder the ratio number_of_population_spikes / number_of_population_neurons
# for every TVB time step, which is already a quantity in the range [0.0, 1.0],
# as long as a neuron cannot fire twice during a TVB time step, i.e.,
# as long as the TVB time step (usually 0.001 to 0.1 ms)
# is smaller than the neurons' refractory time, t_ref (usually 1-2 ms)
tvb_nest_builder.w_spikes_to_tvb = 1000.0

# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------------------
# Configure and build:
tvb_nest_builder.configure()

simulator = tvb_nest_builder.build()

simulator.simulate_spiking_simulator = nest_network.nest_instance.Run  # set the method to run NEST

print("\n\noutput (TVB->NEST coupling) interfaces:\n")
simulator.output_interfaces.print_summary_info_details(recursive=2)

print("\n\ninput (NEST->TVB update) interfaces:\n")
simulator.input_interfaces.print_summary_info_details(recursive=2)


In [ ]:
# print(tvb_nest_model.print_str(detailed_output=True, connectivity=False))

## 4. Configure simulator, simulate, gather results

In [ ]:
# Configure the simulator with the TVB-NEST interface...

# ...and simulate!
t = time.time()

simulator.simulation_length = simulation_length

print("Simulating TVB-NEST...")
nest_network.nest_instance.Prepare()
simulator.configure()
# Adjust simulation length to be an integer multiple of synchronization_time:
simulator.simulation_length = \
    np.ceil(simulator.simulation_length / simulator.synchronization_time) * simulator.synchronization_time
results = simulator.run()
nest_network.nest_instance.Run(nest_network.nest_instance.GetKernelStatus("resolution"))



print("\nSimulated in %f secs!" % (time.time() - t))

In [ ]:
# Clean-up NEST simulation
nest_network.nest_instance.Cleanup()

## 5. Plot results and write them to HDF5 files

In [ ]:
# set to False for faster plotting of only mean field variables and dates, apart from spikes" rasters:
plot_per_neuron = False  
MAX_VARS_IN_COLS = 3
MAX_REGIONS_IN_ROWS = 10
MIN_REGIONS_FOR_RASTER_PLOT = 9

In [ ]:
# If you want to see what the function above does, take the steps, one by one
try:
    # We need framework_tvb for writing and reading from HDF5 files
    from tvb_multiscale.core.tvb.io.h5_writer import H5Writer
    from examples.plot_write_results import write_RegionTimeSeriesXarray_to_h5
    writer = H5Writer()
except:
    writer = False
    
from tvb.contrib.scripts.datatypes.time_series import TimeSeriesRegion
from tvb.contrib.scripts.datatypes.time_series_xarray import TimeSeriesRegion as TimeSeriesXarray

# Put the results in a Timeseries instance
from tvb.contrib.scripts.datatypes.time_series import TimeSeriesRegion

source_ts = TimeSeriesXarray(  # substitute with TimeSeriesRegion fot TVB like functionality
        data=results[0][1], time=results[0][0] - results[0][0][0],
        connectivity=simulator.connectivity,
        labels_ordering=["Time", "State Variable", "Region", "Neurons"],
        labels_dimensions={"State Variable": list(simulator.model.variables_of_interest),
                           "Region": simulator.connectivity.region_labels.tolist()},
        sample_period=simulator.integrator.dt)
source_ts.configure()

t = source_ts.time
    
# Write to file
if writer:
    write_RegionTimeSeriesXarray_to_h5(source_ts, writer,
                                       os.path.join(config.out.FOLDER_RES, source_ts.title)+".h5")
source_ts



In [ ]:
# Plot TVB time series
source_ts.plot_timeseries(plotter_config=plotter.config, 
                          hue="Region" if source_ts.shape[2] > MAX_REGIONS_IN_ROWS else None, 
                          per_variable=source_ts.shape[1] > MAX_VARS_IN_COLS, 
                          figsize=FIGSIZE);

In [ ]:
# # TVB time series raster plot:
# if source_ts.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
#     source_ts.plot_raster(plotter_config=plotter.config, 
#                           per_variable=source_ts.shape[1] > MAX_VARS_IN_COLS,
#                           figsize=FIGSIZE);

In [ ]:
# Focus on the nodes modelled in NEST: 
n_spiking_nodes = len(nest_nodes_ids)
source_ts_nest = source_ts[:, :, nest_nodes_ids]
source_ts_nest.plot_timeseries(plotter_config=plotter.config, 
                               hue="Region" if source_ts_nest.shape[2] > MAX_REGIONS_IN_ROWS else None, 
                               per_variable=source_ts_nest.shape[1] > MAX_VARS_IN_COLS, 
                               figsize=FIGSIZE, figname="Spiking nodes TVB Time Series");

In [ ]:
# # Focus on the nodes modelled in NEST: raster plot
# if source_ts_nest.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
#     source_ts_nest.plot_raster(plotter_config=plotter.config, 
#                                per_variable=source_ts_nest.shape[1] > MAX_VARS_IN_COLS,
#                                figsize=FIGSIZE, figname="Spiking nodes TVB Time Series Raster");

### Interactive time series plot

In [ ]:
# # ...interactively as well
# # For interactive plotting:
# %matplotlib notebook 
# plotter.plot_timeseries_interactive(TimeSeriesRegion().from_xarray_DataArray(source_ts._data,
#                                                                              connectivity=source_ts.connectivity))

### Spiking Network plots

In [ ]:
from tvb_multiscale.core.data_analysis.spiking_network_analyser import SpikingNetworkAnalyser
# Create a SpikingNetworkAnalyzer:
spikeNet_analyzer = \
    SpikingNetworkAnalyser(spikeNet=nest_network,
                           start_time=t[0], end_time=t[-1], 
                           period=simulator.monitors[0].period, transient=transient,
                           time_series_output_type="TVB", return_data=True, 
                           force_homogeneous_results=True, connectivity=simulator.connectivity)

### Plot spikes' raster and mean spike rates and correlations

In [ ]:
# Spikes rates and correlations per Population and Region
spikes_res = \
    spikeNet_analyzer.\
        compute_spikeNet_spikes_rates_and_correlations(
            populations_devices=None, regions=None,
            rates_methods=[], rates_kwargs=[{}],rate_results_names=[],
            corrs_methods=[], corrs_kwargs=[{}], corrs_results_names=[], bin_kwargs={},
            data_method=spikeNet_analyzer.get_spikes_from_device, data_kwargs={},
            return_devices=False
        );

In [ ]:
if spikes_res:
    print(spikes_res["mean_rate"])
    print(spikes_res["spikes_correlation_coefficient"])
    # Plot spikes' rasters together with mean population's spikes' rates' time series
    if plotter:
        plotter.plot_spike_events(spikes_res["spikes"], rates=spikes_res["mean_rate_time_series"], figsize=FIGSIZE) # 
        from tvb_multiscale.core.plot.correlations_plot import plot_correlations
        plot_correlations(spikes_res["spikes_correlation_coefficient"], plotter)

In [ ]:
print("Mean spike rates:")
for pop in spikes_res["mean_rate"].coords["Population"]:
    for reg in spikes_res["mean_rate"].coords["Region"]:
        if not np.isnan(spikes_res["mean_rate"].loc[pop, reg]):
            print("%s - %s: %g" % (pop.values.item().split("_spikes")[0], reg.values.item(), 
                                   spikes_res["mean_rate"].loc[pop, reg].values.item()))

In [ ]:
if spikes_res and writer:
    writer.write_object(spikes_res["spikes"].to_dict(), 
                        path=os.path.join(config.out.FOLDER_RES,  "Spikes") + ".h5");
    writer.write_object(spikes_res["mean_rate"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["mean_rate"].name) + ".h5");
    write_RegionTimeSeriesXarray_to_h5(spikes_res["mean_rate_time_series"], writer,
                                       os.path.join(config.out.FOLDER_RES,
                                                    spikes_res["mean_rate_time_series"].title) + ".h5",
                                       recursive=False);
    writer.write_object(spikes_res["spikes_correlation_coefficient"].to_dict(),
                        path=os.path.join(config.out.FOLDER_RES,
                                          spikes_res["spikes_correlation_coefficient"].name) + ".h5");

### Get  SpikingNetwork mean field variable time series and plot them

In [ ]:
# Continuous time variables' data of spiking neurons
if plot_per_neuron:
    spikeNet_analyzer.return_data = True
else:
    spikeNet_analyzer.return_data = False
spikeNet_ts = \
    spikeNet_analyzer. \
         compute_spikeNet_mean_field_time_series(populations_devices=None, regions=None, variables=None,
                                                 computations_kwargs={}, data_kwargs={}, return_devices=False)
if spikeNet_ts:
    if plot_per_neuron:
        mean_field_ts = spikeNet_ts["mean_field_time_series"]  # mean field
        spikeNet_ts = spikeNet_ts["data_by_neuron"]  # per neuron data
    else:
        mean_field_ts = spikeNet_ts
    if mean_field_ts and mean_field_ts.size > 0:
        mean_field_ts.plot_timeseries(plotter_config=plotter.config, 
                                      per_variable=mean_field_ts.shape[1] > MAX_VARS_IN_COLS)
        if mean_field_ts.number_of_labels > MIN_REGIONS_FOR_RASTER_PLOT:
            mean_field_ts.plot_raster(plotter_config=plotter.config, 
                                      per_variable=mean_field_ts.shape[1] > MAX_VARS_IN_COLS,
                                      linestyle="--", alpha=0.5, linewidth=0.5)
else:
    mean_field_ts = None

In [ ]:
# Write results to file:
if mean_field_ts and writer:
    write_RegionTimeSeriesXarray_to_h5(mean_field_ts, writer,
                                       os.path.join(config.out.FOLDER_RES, mean_field_ts.title) + ".h5", 
                                       recursive=False)

### Compute per neuron spikes' rates times series and plot them

In [ ]:
if spikes_res and plot_per_neuron:
    from tvb.simulator.plot.base_plotter import pyplot
    spikeNet_analyzer.return_data = False
    rates_ts_per_neuron = \
        spikeNet_analyzer. \
            compute_spikeNet_rates_time_series(populations_devices=None, regions=None,
                                               computations_kwargs={}, data_kwargs={},
                                               return_spikes_trains=False, return_devices=False);
    if rates_ts_per_neuron is not None and rates_ts_per_neuron.size:
        # Regions in rows
        row = rates_ts_per_neuron.dims[2] if rates_ts_per_neuron.shape[2] > 1 else None
        if row is None:
            # Populations in rows
            row = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
            col = None
        else:
            # Populations in columns
            col = rates_ts_per_neuron.dims[1] if rates_ts_per_neuron.shape[1] > 1 else None
        pyplot.figure()
        rates_ts_per_neuron.plot(y=rates_ts_per_neuron.dims[3], row=row, col=col, cmap="jet")
        plotter.base._save_figure(figure_name="Spike rates per neuron")
        # del rates_ts_per_neuron # to free memory

### Plot per neuron SpikingNetwork time series

In [ ]:
# Regions in rows
if plot_per_neuron and spikeNet_ts.size:
    row = spikeNet_ts.dims[2] if spikeNet_ts.shape[2] > 1 else None
    if row is None:
        # Populations in rows
        row = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
        col = None
    else:
        # Populations in cols
         col = spikeNet_ts.dims[3] if spikeNet_ts.shape[3] > 1 else None
    for var in spikeNet_ts.coords[spikeNet_ts.dims[1]]:
        this_var_ts = spikeNet_ts.loc[:, var, :, :, :]
        this_var_ts.name = var.item()
        pyplot.figure()
        this_var_ts.plot(y=spikeNet_ts.dims[4], row=row, col=col, cmap="jet", figsize=FIGSIZE)
        plotter.base._save_figure(
            figure_name="Spiking Network variables' time series per neuron: %s" % this_var_ts.name)
    del spikeNet_ts # to free memory

# References

1 Sanz Leon P, Knock SA, Woodman MM, Domide L, <br>
  Mersmann J, McIntosh AR, Jirsa VK (2013) <br>
  The Virtual Brain: a simulator of primate brain network dynamics. <br>
  Frontiers in Neuroinformatics 7:10. doi: 10.3389/fninf.2013.00010 <br>
  https://www.thevirtualbrain.org/tvb/zwei <br>
  https://github.com/the-virtual-brain <br>

2 Ritter P, Schirner M, McIntosh AR, Jirsa VK (2013).  <br>
  The Virtual Brain integrates computational modeling  <br>
  and multimodal neuroimaging. Brain Connectivity 3:121–145. <br>

3 Jordan, Jakob; Mørk, Håkon; Vennemo, Stine Brekke;   Terhorst, Dennis; Peyser, <br>
  Alexander; Ippen, Tammo; Deepu, Rajalekshmi;   Eppler, Jochen Martin; <br>
  van Meegen, Alexander;   Kunkel, Susanne; Sinha, Ankur; Fardet, Tanguy; Diaz, <br>
  Sandra; Morrison, Abigail; Schenck, Wolfram; Dahmen, David;   Pronold, Jari; <br>
  Stapmanns, Jonas;   Trensch, Guido; Spreizer, Sebastian;   Mitchell, Jessica; <br>
  Graber, Steffen; Senk, Johanna; Linssen, Charl; Hahne, Jan; Serenko, Alexey; <br>
  Naoumenko, Daniel; Thomson, Eric;   Kitayama, Itaru; Berns, Sebastian;   <br>
  Plesser, Hans Ekkehard <br>
  NEST is a simulator for spiking neural network models that focuses <br>
  on the dynamics, size and structure of neural systems rather than on <br>
  the exact morphology of individual neurons. <br>
  For further information, visit http://www.nest-simulator.org. <br>
  The release notes for this release are available at  <br>
  https://github.com/nest/nest-simulator/releases/tag/v2.18.0 <br>

4 Baladron, J., Nambu, A., & Hamker, F. H. (2019). <br>
  The subthalamic nucleus‐external globus pallidus loop biases <br>
  exploratory decisions towards known alternatives: A neuro‐computational study. <br>
  European Journal of Neuroscience, 49:754–767. https://doi.org/10.1111/ejn.13666 <br>
  
5 Maith O, Villagrasa Escudero F, Ülo Dinkelbach H, Baladron J, <br>
  Horn, A, Irmen F, Kühn AA, Hamker FH (2020).<br>
  A computational model‐based analysis of basal ganglia pathway changes <br>
  in Parkinson’s disease inferred from resting‐state fMRI <br>
  European Journal of Neuroscience, 00:1–18. https://doi.org/10.1111/ejn.14868
  